# Azure AI Memory Store with LangGraph

This notebook demonstrates how to use `AzureAIMemoryStore` to persist and retrieve
agent memories using the Azure AI Projects SDK V2 and LangGraph's `BaseStore` interface.

## Prerequisites

1. A Microsoft Foundry project.
2. A deployed chat model (e.g., `gpt-4.1`).
3. A deployed embedding model (e.g., `text-embedding-3-small`).
4. An existing Azure AI memory store in your project.
5. Environment variables set:

   - `AZURE_AI_PROJECT_ENDPOINT`
   - `MEMORY_STORE_NAME`

6. Install dependencies:

   ```
   pip install 'azure-ai-projects>=2.0.0b1' --pre
   pip install langchain-azure-ai langgraph
   ```

In [ ]:
import os

os.environ["AZURE_AI_PROJECT_ENDPOINT"] = (
    "https://<resource>.services.ai.azure.com/api/projects/<project>"
)
os.environ["MEMORY_STORE_NAME"] = "my-memory-store"

## Create a memory store in Azure AI Projects

The memory store must exist before we can use `AzureAIMemoryStore`.
The code below creates one if it does not already exist.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    MemoryStoreDefaultDefinition,
    MemoryStoreDefaultOptions,
)
from azure.core.exceptions import ResourceExistsError
from azure.identity import DefaultAzureCredential

endpoint = os.environ["AZURE_AI_PROJECT_ENDPOINT"]
memory_store_name = os.environ["MEMORY_STORE_NAME"]

project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential(),
)

try:
    definition = MemoryStoreDefaultDefinition(
        chat_model="gpt-4.1",
        embedding_model="text-embedding-3-small",
        options=MemoryStoreDefaultOptions(
            user_profile_enabled=True,
            chat_summary_enabled=True,
        ),
    )
    memory_store = project_client.beta.memory_stores.create(
        name=memory_store_name,
        description="LangGraph memory store demo",
        definition=definition,
    )
    print(f"Created memory store: {memory_store.name}")
except ResourceExistsError:
    print(f"Memory store '{memory_store_name}' already exists")

## Initialise AzureAIMemoryStore

`AzureAIMemoryStore` wraps the Azure AI memory store and implements the LangGraph
`BaseStore` interface so it can be passed directly to any LangGraph graph or agent.

In [ ]:
from langchain_azure_ai.stores import AzureAIMemoryStore

store = AzureAIMemoryStore(
    project_client=project_client,
    memory_store_name=memory_store_name,
)

print("AzureAIMemoryStore ready")

## Put – store memories

Use `store.put(namespace, key, value)` to persist a memory.  
The namespace is a tuple of strings that groups related memories (e.g. by user ID).  
The value is any JSON-serialisable dictionary.

In [ ]:
# Store a user preference
store.put(
    ("users", "alice"),
    "preferences",
    {"theme": "dark", "language": "en", "coffee": "dark roast"},
)

# Store a fact about the user
store.put(
    ("users", "alice"),
    "profile",
    {"name": "Alice", "location": "Seattle", "role": "engineer"},
)

print("Memories stored")

## Get – retrieve a specific memory by key

In [ ]:
item = store.get(("users", "alice"), "preferences")
if item:
    print(f"Found: key={item.key}, value={item.value}")
else:
    print("Not found")

## Search – semantic search over memories

Use `store.search(namespace_prefix, query=...)` to find memories semantically
relevant to a natural-language query.

In [ ]:
results = store.search(("users", "alice"), query="What coffee does Alice prefer?")
for result in results:
    print(f"key={result.key}, value={result.value}")

## Delete – remove memories for a namespace

Passing `value=None` to `put()` deletes all memories for the given namespace scope.

In [ ]:
store.delete(("users", "alice"), "preferences")
print("Deleted preferences namespace scope")

## Use with a LangGraph agent

Pass the store directly to any LangGraph graph as the `store` parameter.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, MessagesState, StateGraph


def agent_node(state: MessagesState, store: AzureAIMemoryStore) -> dict:  # type: ignore[type-arg]
    """Simple agent that retrieves memories and returns a greeting."""
    user_id = "alice"
    memories = store.search(
        ("users", user_id),
        query=state["messages"][-1].content,
    )
    memory_context = "\n".join(
        f"- {m.key}: {m.value}" for m in memories
    )
    from langchain_core.messages import AIMessage
    return {
        "messages": [
            AIMessage(
                content=f"I found the following memories:\n{memory_context}"
            )
        ]
    }


builder = StateGraph(MessagesState)
builder.add_node("agent", agent_node)
builder.add_edge(START, "agent")
builder.add_edge("agent", END)

graph = builder.compile(store=store, checkpointer=MemorySaver())

response = graph.invoke(
    {"messages": [HumanMessage(content="What are Alice's coffee preferences?")]},
    config={"configurable": {"thread_id": "thread-1"}},
)

for msg in response["messages"]:
    msg.pretty_print()

## Clean up

In [ ]:
project_client.beta.memory_stores.delete(memory_store_name)
print(f"Memory store '{memory_store_name}' deleted")